# Fitting Thermoelectric data
Models and data are from Danny/Kedar.

## Import Modules, Functions, and Data

`functions.py` has the Python implementations of all the helper functions (I used a previously written package, `fdint`, for the Fermi-Dirac integrals).

In [1]:
import numpy as np
from functions import *

In [2]:
celldata = {}
celldata['xdata'] = np.loadtxt('xdata.csv',delimiter=',')
celldata['ydata'] = np.loadtxt('ydata.csv',delimiter=',')
celldata['n'] = 40

## Validate Python implementation
I did a test evaluation in Matlab and Python with the same input parameters. Let's import the results and compare to make sure we're getting the same thing.

In [3]:
test_in = [-2.499946233286e1,1.833885014595e-3,-2.2588468610036e-3,8.6217332036812e-4]
test_y,test_S,test_Rou=tefunnew(celldata,test_in)
matlab_y = np.loadtxt('matlab_y.csv',delimiter=',')
matlab_S = np.loadtxt('matlab_S.csv',delimiter=',')
matlab_Rou = np.loadtxt('matlab_Rou.csv',delimiter=',')

In [4]:
y_pct_diff=(test_y-matlab_y)/matlab_y
print('There is an average of a %.2f%% difference (with a standard deviation of %.2f%%) between the Matlab and Python implementations in the y output.'%(round(100.0*np.mean(y_pct_diff),2),round(100.0*np.std(y_pct_diff),2)))

There is an average of a -3.27% difference (with a standard deviation of 0.05%) between the Matlab and Python implementations in the y output.


In [5]:
S_pct_diff=(test_S-matlab_S)/matlab_S
print('There is an average of a %.2f%% difference (with a standard deviation of %.2f%%) between the Matlab and Python implementations in the S output.'%(round(100.0*np.mean(S_pct_diff),2),round(100.0*np.std(S_pct_diff),2)))

There is an average of a -1.65% difference (with a standard deviation of 0.03%) between the Matlab and Python implementations in the S output.


In [6]:
Rou_pct_diff=(test_Rou-matlab_Rou)/matlab_Rou
print('There is an average of a %.3f%% difference (with a standard deviation of %.3f%%) between the Matlab and Python implementations in the Rou output.'%(round(100.0*np.mean(Rou_pct_diff),3),round(100.0*np.std(Rou_pct_diff),3)))

There is an average of a -0.051% difference (with a standard deviation of 0.002%) between the Matlab and Python implementations in the Rou output.


Okay, so the differences aren't nothing, but they're small enough that I think we can work with them.

## Fitting with Bayesim
Now let's do a fit to the data using the grid approach implemented in the `bayesim` code.

In [7]:
import sys
sys.path.append('../../')
import bayesim.model as bym
import bayesim.param_list as byp
import functions as tefcns # model functions implemented in a separate file to keep this notebook tidy

First, we set up the list of parameters to be fit and their ranges.

In [8]:
fp = byp.param_list()
"""
fp.add_fit_param(name='P0', val_range=[1e-34,1e-20], spacing='log', length=28, units='sec.')
fp.add_fit_param(name='fs', val_range=[-1,2], length=21, units='eV')
fp.add_fit_param(name='r', val_range=[-1,2], length=21)
fp.add_fit_param(name='Z', val_range=[-10,10], length=20)
"""
fp.add_fit_param(name='P0', val_range=[1e-34,1e-20], spacing='log', length=5, units='sec.')
fp.add_fit_param(name='fs', val_range=[-1,2], length=5, units='eV')
fp.add_fit_param(name='r', val_range=[-1,2], length=5)
fp.add_fit_param(name='Z', val_range=[-10,10], length=5)


Next, define the experimental conditions.

In [9]:
ec = ['T','R','n']

Now, set up the `bayesim.model` object. All we need to feed in are the parameters, experimental conditions, and name of the output variable.

In [10]:
m = bym.model(params=fp,ec=ec,output_var='P')

The next thing to do is to attach the observed data. I reformatted it to work with `bayesim` and saved an HDF5 file. You can see the format in the Excel sheet `TE_expt_data.xlsx`.

In [11]:
m.attach_observations(fpath='TE_expt_data.h5')

Identified experimental conditions as ['n', 'T', 'R']. If this is wrong, rerun and explicitly specify them with attach_ec (make sure they match data file columns) or remove extra columns from data file.


Next, we attach the model. I precomputed the values on the initial grid so we can just attach the data output from that rather than attaching a function, which would cause it to be directly computed here and that would take awhile.

In [12]:
#import pandas as pd
import time
start = time.time()
"""
param_pts = m.probs.points[m.probs.points['new']==True][m.param_names]
param_inds = range(len(param_pts))
ec_inds = range(len(m.ec_pts))
columns = m.param_names + m.ec_names
pts = []
for ppt in param_pts.iterrows():
    for ecpt in m.ec_pts.iterrows():
        pts.append(list(ppt[1])+list(ecpt[1]))
sim_pts = pd.DataFrame(data=pts,columns=columns)
"""
m.list_model_pts_to_run('./sim_list.h5')
print(time.time()-start)

Making the list of points...
Done making list (66.530000.2 seconds). Now saving the list to HDF5...
67.2952752113


In [13]:
len(m.ec_pts)

870

In [14]:
len(m.probs.points)

625

In [17]:
list(m.ec_pts.columns)

['n', 'T', 'R']

In [19]:
import deepdish as dd
sim_list = dd.io.load('sim_list.h5')

In [20]:
sim_list.head()

,P0,fs,r,Z,n,T,R
0,2.511886e-33,-0.7,-0.7,-8.0,40.0,10.0,1.00000
1,2.511886e-33,-0.7,-0.7,-8.0,40.0,10.0,1.33333
2,2.511886e-33,-0.7,-0.7,-8.0,40.0,10.0,1.66667
3,2.511886e-33,-0.7,-0.7,-8.0,40.0,10.0,2.00000
4,2.511886e-33,-0.7,-0.7,-8.0,40.0,10.0,2.33333


In [21]:
max(m.probs.points.index)

624

In [22]:
m.ec_names,m.param_names

(['n', 'T', 'R'], ['P0', 'fs', 'r', 'Z'])

In [28]:
len(m.ec_pts)

870

In [25]:
import pandas as pd
pd.DataFrame.from_records(data=m.obs_data.groupby(m.ec_names).groups.keys(),columns=m.ec_names).round(m.ec_tol_digits).sort_values(m.ec_names).reset_index(drop=True)

,n,T,R
0,40,10,1.00000
1,40,10,1.33333
2,40,10,1.66667
3,40,10,2.00000
4,40,10,2.33333
5,40,10,2.66667
6,40,10,3.00000
7,40,10,3.33333
8,40,10,3.66667
9,40,10,4.00000
